In [5]:
import logging
import ipywidgets as w

import logging.config
import yaml

with open('file.yml', 'r') as f:
    config = yaml.safe_load(f.read())
    logging.config.dictConfig(config)

# Get the logger, console_logger, which will print to the console.
#logger = logging.getLogger('console_logger')



from myadd import add
from myminus import minus



# REF
# ---
# https://github.com/jupyter-widgets/ipywidgets/pull/2268/commits/bced184b2486b1e6af61d2a3848ed755dec2071b#diff-a9f5e21507fb26263452386f1cb42ddbbaa7b7e26ccc5b1348fec1e205245b70
# todo: use core ipywidgets when the above is merged

def div(a, b):
    try:
        c = a / b
        logger.debug("c = %s", c)
        return c
    except Exception as e:
        logger.error("Exception occurred", exc_info=True)

class Output(w.Output):
    def register_logger(self, logger, *args, **kwargs):
        """Registers a handler to given logger to send output to output widget"""

        import logging

        class WidgetLogger(logging.Handler):
            """ Class to implement a logging interface that outputs to the
             Output widget"""

            # have a class member to store the existing logger
            logger_instance = logging.getLogger("__name__")

            def __init__(self, output_widget, *args, **kwargs):
                # Initialize the Handler
                logging.Handler.__init__(self, *args)
                
                # formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
                # self.setFormatter(formatter)

                # save outer_instance
                self.output = output_widget

                # optional take format
                # setFormatter function is derived from logging.Handler
                for key, value in kwargs.items():
                    if "{}".format(key) == "format":
                        self.setFormatter(value)

                # make the logger send data to this class
                self.logger_instance.addHandler(self)

            def emit(self, record):
                """ Overload of logging.Handler method """

                record = self.format(record)
                self.output.outputs = ({'name': 'stdout',
                                                'output_type': 'stream',
                                                'text': (record + '\n')},) + self.output.outputs

        logger.addHandler(WidgetLogger(self, *args, **kwargs))


class Ui(w.VBox):
    def __init__(self, loggers: list):
        self.a = w.IntText()
        self.b = w.IntText()
        self.c = w.IntText()
        self.out = Output()
        [self.out.register_logger(logger, format=logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')) for logger in loggers]
        self.acc = w.Accordion(children=[self.out])
        self.c.disabled = True
        self.div = w.Button(description="operate")
        self.div.on_click(self.on_div)
        super().__init__([self.a, self.b, self.c, self.div, self.acc])

    def on_div(self, b):
        _ = add(self.a.value, self.b.value)
        _ = minus(_, self.a.value)
        self.c.value = div(_, self.b.value)


logger = logging.getLogger(__name__)
logger.info('This is a debug message')
loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
loggers = [l for l in loggers if "my" in l.name ] + [logger]
ui = Ui(loggers=loggers)
ui

Ui(children=(IntText(value=0), IntText(value=0), IntText(value=0, disabled=True), Button(description='operate'…

In [ ]:
loggi

In [9]:
# [logging.getLogger(name) for name in logging.root.manager.loggerDict]
loggers


[<Logger myadd (WARNING)>,
 <Logger myminus (WARNING)>,
 <Logger __main__ (WARNING)>]

In [4]:
loggers

[<Logger myadd (WARNING)>,
 <Logger myminus (WARNING)>,
 <Logger __main__ (WARNING)>]

In [8]:
debug_view = w.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = w.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [10]:
# https://ipyw.readthedocs.io/en/7.7.1/examples/Output%20Widget.html

import ipywidgets as w
import logging



In [11]:
handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')

'__main__'